In [1]:
import urllib.request
import os
import tarfile #处理压缩打包文件

In [2]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
#这个类用来对文本中的词进行统计计数，生成文档词典，以支持基于词典位序生成文本的向量表示
from tensorflow.keras.preprocessing.text import Tokenizer

import numpy as np

In [3]:
#数据预处理 去除符号
import re
def rm_tags (text): #传递文本
    re_tag = re.compile(r'<[^>]+>') #清除标签，符号等  例如<a></a>（因为是从网页上找的影评）
    return re_tag.sub('',text)  #只输出内容

In [4]:
#导入数据
import os
def read_files(filetype):      #train/test都有
    path = "D:/filp/program/Writer.git/data/imdb/"  #路径
    file_list=[]   #存储读取的所有正负面语料文件名字
    
    positive_path=path+filetype+"/pos/"  #寻找所有正面评价语料
    for f in os.listdir(positive_path):
        file_list+=[positive_path+f]        #将正面评价路径附加到存储列表里

    negative_path=path+filetype+"/neg/"   #负面
    for f in os.listdir(negative_path):
        file_list+=[negative_path+f]         #负面路径

    print('read',filetype,'files:',len(file_list))
        
    all_labels=([1] * 12500 + [0] * 12500)  #1为正面  0为负面

    all_texts = []

    for fi in file_list:
        with open(fi,encoding='utf8') as file_input:   #分流的方式打开文件
            all_texts += [rm_tags(" ".join(file_input.readlines()))]  #一次读取一行并清除标签

    return all_labels,all_texts


In [5]:
y_train,train_text=read_files("train" )

read train files: 25000


In [6]:
y_test,test_text=read_files("test" )

read test files: 25000


In [ ]:
#查看正面评价第一个
train_text[0]

In [ ]:
#检查是不是1
y_train[0]

In [ ]:
#查看负面评价第一个
train_text[12500]

In [ ]:
#检查是不是0
y_train[12500]

In [ ]:
#先读取所有文章建立字典,限制字典的数量为nb_ words=2000
#将文本分解成不同的单元（分为单词，若分成字符没有意义）成为标记
#词袋模型（已经淘汰）：分词之后，将标记放到无序的集合中，而不是一个列表或序列，即标记没有特定的顺序
#单词转向量:词嵌入(低维度、密集、从数据中学习，、0neHot编码(高维、稀疏、需要编码)

#使用Keras的Tokenizer模块实现转换。当我们创建了一个Tokenizer对象后，使用该对象的fit_on_texts()函数，
#可以将输入的文本中的每个词编号，编号是根据词频的，词频越大，编号越小。使用word_index属性可以看到每次词对应的编码。
token = Tokenizer(num_words=2000) 
token.fit_on_texts(train_text) #分词，之后计算数量再排序，取前num_woeds个作为字典元素

In [ ]:
#Tokenizer橱性
#fit on texts 读取多少文章
print(token.document_count)

In [ ]:
#统计各个单词出现的频率
print(token.word_index)

In [ ]:
#将每一篇文章的中的文字转换为一连串的数字
x_train_seq = token.texts_to_sequences(train_text)#将多个文档转换为word下标的向量形式
x_test_seq  = token.texts_to_sequences(test_text)

In [ ]:
print(train_text[0])

In [ ]:
#每个单词对应一个数字 表示该单词再词典中的位置
print(x_train_seq[0])

In [ ]:
#让转换后的数字长度相同#文章内的文字，转换为数字后，每一篇的文章所产生的数字长度都不同，
#因为之后需要进行类神经网络的训练，所以每篇文章所产生的数字长度必须相同
#以下列程序代码为例max1en=100，所以每一篇文章转换为数字都必须为100
x_train = sequence.pad_sequences(x_train_seq,maxlen=100)#让每句数字影评长度相同
x_test  = sequence.pad_sequences(x_test_seq, maxlen=100)

In [ ]:
#如果文章转换的数字数量大于maxlen, pad_sequences处理后，会切掉前面的数字
#因为前面的话往往是铺垫，中后才是表达情感的部分
print('before pad_sequences length=',len(x_train_seq[0]))
print(x_train_seq[0])

In [ ]:
print('after pad_sequences length=',len(x_train[0]))
print(x_train[0])

In [ ]:
#如果文章转成数字不足100, pad seguences处理后，前面会加0
print('before pad_sequences length=',len(x_train_seq[3]))
print(x_train_seq[3])

In [ ]:
print('after pad_sequences length=',len(x_train[3]))
print(x_train[3])

In [ ]:
#建立模型 引入嵌入层  Embedding:词嵌入
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.layers import Embedding#从稀疏矩阵到密集矩阵的过程，叫做embedding

In [ ]:
model = Sequential()

In [ ]:
#添加词嵌入 即单词转向量
model.add(Embedding(output_dim=32,    #每个单词转换成向量的长度
                    input_dim=2000,   #对应词典大小
                    input_length=100))#对应处理后的长度

model.add(Dropout(0.2)) #丢弃层，尽可能降低过拟合，每次只选择80%

#下面的警告是Tensorflow版本问题  提示有的语句新版本不适用 但是换版本的话程序运行不了 和python有冲突

In [ ]:
model.add(Flatten())  #添加一个平坦层  把多维向量一维化

In [ ]:
model.add(Dense(units=256,activation='relu')) #units :设置神经元数量256个 激活函数是rulu
model.add(Dropout(0.2)) #丢弃层，尽可能降低过拟合，每次只选择80%

In [ ]:
#输出模型训练结果
model.add(Dense(units=1,
                activation='sigmoid'))

In [ ]:
#查看模型
model.summary()
#嵌入层  参数64000=32*2000；
#隐藏层  819456=3200（3201）*256

In [ ]:
#优化模型 归一化处理
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])

In [ ]:
#训练模型
#epochs                                                  批次数               80%训练   20%做验证
train_history =model.fit(x_train, y_train,batch_size=100,epochs=10,verbose=2,validation_split=0.2)

#loss：损失  acc:精度   val_loss验证损失误差   val_acc验证精度

In [ ]:
%matplotlib inline 
import matplotlib.pyplot as plt
def show_train_history(train_history,train,validation):#训练集 和验证集
    plt.plot(train_history.history[train])
    plt.plot(train_history.history[validation])
    plt.title('Train History')
    plt.ylabel(train)
    plt.xlabel('Epoch')
    plt.legend(['train','validation'],loc='upper left')
    plt.show()

In [ ]:
#keras库老版本中的参数不是accuracy，而是acc
#keras库老版本中的参数不是val_accuracy,而是val_acc
show_train_history(train_history,'acc','val_acc')   
#精度表
#蓝色为训练   黄色为验证

In [ ]:
#打印损失和误差
show_train_history(train_history,'loss','val_loss')
#损失表
#蓝色为训练   黄色为验证

In [ ]:
#评估模型真正的准确率
scores = model.evaluate(x_test,y_test,verbose=1)
scores[1]
#训练的时候0.9922   评估的时候只有0.808

In [ ]:
#预测概率
probility=model.predict(x_test)

In [ ]:
#打印前10个文本的评估结果
probility[:10]

In [ ]:
#打印12500到12510文本的评估结果  有的判断有错误
for p in probility[12500:12510]:
    print(p)

In [ ]:
#预测结果0或1  验证集
predict=model.predict_classes(x_test)

In [ ]:
#前10个
predict[:10]

In [ ]:
predict_classes=predict.reshape(-1)#变成一维的 横向显示
predict_classes[:10]

In [ ]:
#查看预测结果   将0，1转换成较为直观的中文结果
SentimentDict={1:'正面的',0:'负面的'}
def display_test_Sentiment(i):
    print(test_text[i])                                 #打印文本
    print('标签1abel:',SentimentDict[y_test[i]],        #文本标签
          '预测结果:',SentimentDict[predict_classes[i]])#预测结果


In [ ]:
#查看正面评价第三个文本的预测结果
display_test_Sentiment(2)

In [ ]:
predict_classes[12500:12510]

In [ ]:
#查看负面评价第五个的预测结果
display_test_Sentiment(12504)

In [ ]:
#预测新的影评
input_text='''
this is awesome!!! there is no partnership quite like Errol, and Olivia. there love is genuine! I'm 24, yet this flick is as captivating now as I'm sure it was 60 years ago. Raoul Walsh is an under-rated genius, his direction is so sweeping, so broad, yet so intimate. the last scene between colonel custer (Flynn), and his wife (de havilland), almost brought me to tears (Not easy for a 24yr old guy!!), its so heart-wrenching. there is also a deep Christian message implicit here, the faith Custer has in taking your glory with you, and the trust, and fidelity of his wife to the extent of letting him go, in order that he fulfils his moral duty to protect the innocent civilians from certain massacre. there is no movie that deals with these issues quite like this. a must-see for anyone who wants to look at this defining moment in American, and military history, from the inside. patriotic, for all the right reasons. i knew Errol Flynn was a star, and De havilland was a screen legend-this only confirms my suspicions that they are among the very greatest!
'''

In [ ]:
#下面的工作也可以通过def定义为一个函数一步处理完毕
#需要对输入的文本做预处理
input_seq = token.texts_to_sequences([input_text]) 

In [ ]:
#文本数字化长度
len(input_seq[0])

In [ ]:
#截取为长度100的数据
pad_input_seq = sequence.pad_sequences(input_seq , maxlen=100)

In [ ]:
len(pad_input_seq[0])

In [ ]:
predict_result=model.predict_classes(pad_input_seq)

In [ ]:
predict_result[0][0]

In [ ]:
SentimentDict[predict_result[0][0]]